In [14]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# 讀取數據 
df = pd.read_csv('ecommerce_customer_behavior_5000.csv')

sns.set_theme(style="whitegrid", palette="muted")
print("Data loaded successfully", len(df), "orders.")

Data loaded successfully 5000 orders.


In [18]:
# Structural Audit
print("structural Audit: Column Data Types")
df.info()
# Data Integrity
print("data integrity check: missing value")
missing_values = df.isnull().sum()
if missing_values.sum() == 0:
    print("No missing values detected. Dataset is clean.")
else:
    print(missing_values[missing_values > 0])


structural Audit: Column Data Types
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   order_id            5000 non-null   int64         
 1   customer_age        5000 non-null   int64         
 2   customer_gender     5000 non-null   category      
 3   product_category    5000 non-null   object        
 4   payment_method      5000 non-null   category      
 5   order_value_usd     5000 non-null   float64       
 6   delivery_time_days  5000 non-null   int64         
 7   customer_rating     5000 non-null   float64       
 8   returned            5000 non-null   object        
 9   order_date          5000 non-null   datetime64[ns]
 10  returned_status     2475 non-null   float64       
dtypes: category(2), datetime64[ns](1), float64(3), int64(3), object(2)
memory usage: 361.8+ KB
data integrity check: missing val

In [21]:
# Data Transformation
for col in ['customer_gender', 'product_category', 'payment_method', 'returned']:
    unique_values = df[col].unique()
    print(f"Column '{col}' unique values: {unique_values}")
    
# Numerical outlier
print("\n--- [Audit] Logical Range Check ---")
print(f"Delivery Time Range:' {df['delivery_time_days'].min()} to {df['delivery_time_days'].max()} days")
invalid_ratings = df[(df['customer_rating']<1) | (df['customer_rating']>5)].shape[0]

print(f"Out-of-range Rating: {invalid_ratings}")

# duplicate data

duplicates = df.duplicated(subset=['order_id']).sum()
print(f"Duplicate Orders:{duplicates}")

Column 'customer_gender' unique values: ['Other', 'Female', 'Male']
Categories (3, object): ['Female', 'Male', 'Other']
Column 'product_category' unique values: ['Home & Living' 'Fashion' 'Sports' 'Electronics' 'Groceries' 'Beauty']
Column 'payment_method' unique values: ['Wallet', 'Cash on Delivery', 'Debit Card', 'Credit Card', 'UPI']
Categories (5, object): ['Cash on Delivery', 'Credit Card', 'Debit Card', 'UPI', 'Wallet']
Column 'returned' unique values: ['No' 'Yes']

--- [Audit] Logical Range Check ---
Delivery Time Range:' 1 to 14 days
Out-of-range Rating: 0
Duplicate Orders:0


In [24]:
# String Standardisation and Data Transformation
# DD/MM/YYYY
df['returned'] = df['returned'].str.strip().str.capitalize()
df['order_date'] = pd.to_datetime(df['order_date'],format ='mixed')
# Yes=1; No=0
df['returned_status'] = df['returned'].map({'Yes':1,'No':0})

# typed transferred
cols_to_category = ['customer_gender', 'product_category', 'payment_method']
for col  in cols_to_category:
    df[col] = df[col].astype('category')

df['product_category'] = df['product_category'].str.strip()

print("\n Data Transformation complete.")
df.info()



 Data Transformation complete.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   order_id            5000 non-null   int64         
 1   customer_age        5000 non-null   int64         
 2   customer_gender     5000 non-null   category      
 3   product_category    5000 non-null   object        
 4   payment_method      5000 non-null   category      
 5   order_value_usd     5000 non-null   float64       
 6   delivery_time_days  5000 non-null   int64         
 7   customer_rating     5000 non-null   float64       
 8   returned            5000 non-null   object        
 9   order_date          5000 non-null   datetime64[ns]
 10  returned_status     5000 non-null   int64         
dtypes: category(2), datetime64[ns](1), float64(2), int64(4), object(2)
memory usage: 361.8+ KB


In [28]:
# Execute Summary Metrics
total_return_rate: float = df['returned_status'].mean() * 100
avg_order_rate: float = df['order_value_usd'].mean()

print(f" Overall Return Rate: {total_return_rate:.2f}%")
print(f" Ave Order Value: {avg_order_rate:.2f}")

# Return Rate by Category
category_returns = df.groupby('product_category')['returned_status'].mean().sort_values(ascending=False) * 100

print("Return Rate by Category")
print(category_returns)

 Overall Return Rate: 50.50%
 Ave Order Value: 749.67
Return Rate by Category
product_category
Beauty           52.286049
Groceries        51.373955
Home & Living    51.048089
Sports           50.646298
Electronics      49.309912
Fashion          48.296122
Name: returned_status, dtype: float64
